# AutoGenU for Jupyter

An Automatic C++ Code Generator for Nonlinear Model Predictive Control (NMPC)  

---  

# Generate C++ codes of NMPC model  

## Import modules

In [ ]:
from sympy import *
from autogenu import symbolic_funcs as symfunc
from autogenu import cpp_generator as gencpp

## Set dimensions
`dimx`: dimension of the state vector $x$   
`dimu`: dimension of the control input vector $u$  
`dimc`: dimension of the constraints $C(x, u) = 0$  

In [ ]:
dimx = 
dimu = 
dimc = 

## Generate t, x, u, and lmd, necessary variables for the formulation of the optimal control problem
`t`: time parameter $t$  
`x`: the state vector $x$  
`u`: the control input vector $u$  
`lmd`: the Lagrange multiplier vector for the state equation $\lambda$

In [ ]:
t = Symbol('t')  
x = symbols('x[0:%d]' %(dimx))  
u = symbols(f'u[0:%d]' %(dimu+dimc))
lmd = symbols(f'lmd[0:%d]' %(dimx))

## Define variables used in the state equation, constraints, and cost function
Define a scalar variable whose name is "var" as  
`var = Symbol('var') `

Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = symbols('var_1, var_2, ..., var_n')`

Define a vector whose name is "vec" and dimension is dim_vec as  
`vec = symbols(f'vec[0:{dim_vec}]')`

In [ ]:
# Define user variables used in the state function here
 = symbols('')

# You can also define functions used  in the state function or in the cost funciton 

## Define weight parameters used in the stage cost and the terminal cost
`q`: weight array for the state in the stage cost  
`r`: weight array for the control input in the stage cost  
`q_terminal`: weight array for the state in the terminal cost  
`x_ref`: the reference value of the state

In [ ]:
q = symbols(f'q[0:{dimx}]')
r = symbols(f'r[0:{dimu+dimc}]')
q_terminal = symbols(f'q_terminal[0:{dimx}]')
x_ref = symbols(f'x_ref[0:{dimx}]')

## Define the state equation, constraints, the stage cost, and the terminal cost
`fxu`: state equation $ f(t, x, u)$  
`Cxu`: equality constraisnts $C(t, x, u) $  
`L`: stage cost $L(t, x, u)$  
`phi`: terminal cost $\phi (t, x)$  
Note: array indices start with 0

In [ ]:
# Define the state equation
fxu = []

# Define the constraints (if dimc > 0)
Cxu = []

# Define the stage cost
L = sum(q[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 + sum(r[i] * u[i]**2 for i in range(dimu))/2 

# Define the terminal cost
phi = sum(q_terminal[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2

## Generate the optimality conditions
`hamiltonian`: $H(t, x, u, \lambda) = L(t, x, u) + \lambda^{\rm T} f(t, x, u)$  
`hx`: partial derivartive of the hamiltonian $H (t, x, u, \lambda)$ with respect to $x$, $\left(\frac{\partial H}{\partial x} \right)^{\rm T} (t, x, u, \lambda)$    
`hu`: partial derivartive of the hamiltonian $H (t, x, u, \lambda)$ with respect to $u$, $\left(\frac{\partial H}{\partial u} \right)^{\rm T}, (t, x, u, \lambda)$    
`phix`: partial derivative of the terminal cost $\phi(t, x)$ with respect to $x$, $\left(\frac{\partial \phi}{\partial x} \right)^{\rm T} (t, x)$

In [ ]:
if(dimc > 0):
    hamiltonian = L + symfunc.dot_product(lmd, fxu) + sum(u[dimu+i] * Cxu[i] for i in range(dimc))
else:
    hamiltonian = L + symfunc.dot_product(lmd, fxu) 

phix = symfunc.diff_scalar_func(phi, x)
hx = symfunc.diff_scalar_func(hamiltonian, x)
hu = symfunc.diff_scalar_func(hamiltonian, u)

## Symplify phix, hx, and hu
Note: if `hx` and `hu` is too complicated, it takes too much time to simplify them

In [ ]:
phix = simplify(phix)
hx = simplify(hx)
hu = simplify(hu)

## Set Parameters
set all parameters used in the state equation, constraints, and the cost function

In [ ]:
# scalar parameters
scalar_params = []

# array parameters
array_params = [['q', dimx, '{}'], 
                ['r', dimu, '{}'], 
                ['q_terminal', dimx, '{}'], 
                ['x_ref', dimx, '{}']]


## Generate C++ codes of NMPC model
generate `nmpc_model.hpp` and `nmpc_model.cpp` in a directory of `model_name`

In [ ]:
model_name = ""

gencpp.make_model_dir(model_name)
gencpp.generate_cpp(dimx, dimu, dimc, fxu, Cxu, phix, hx, hu, model_name)
gencpp.generate_hpp(dimx, dimu, dimc, scalar_params, array_params, model_name)

---  
# Generate C++ codes for numerical simulation  

## Import modules

In [ ]:
from autogenu import solver_params as slvprm
from autogenu import initialization_params as iniprm
from autogenu import simulation_params as simprm
from autogenu import cpp_executor as cppexe
from autogenu import simulation_plottor as simplt
init_printing()

## Set solvers  
set which solvers you use  in `solver_index`
  
1. The continuation/GMRES method (the original C/GMRES method, single shooting)
2. The multiple shooting based C/GMRES method
3. The multiple shooting based C/GMRES method with condensing of variables with respect to the constraints on the saturation function on the control input

In [ ]:
solver_index = 

## Set saturaions on the control input if you choose `solver_index = 3`
- saturation on the control input: $u_{i, {\rm min}} \leq u_i \leq u_{i, {\rm max}}$  
$u_i \in \mathbb{R}$ : a constrained component of the control input $u$  
- transformed equality constraint: $(u_i - \frac{u_{i, {\rm max}} + u_{i, {\rm min}}}{2})^2 - ( \frac{u_{i, {\rm max}} - u_{i, {\rm min}}}{2})^2 + {u_d}_i ^2 = 0$  
${u_d}_i \in \mathbb{R}$ : a dummy input for the transformation of the saturation into the equality constraint  
- additional term in the stage cost $L(x, u)$ with respect to the saturation of $u_i$: $- {r_d}_i {u_d}_i + \frac{1}{2} {r_q}_i {u_d}_i ^2$  
   ${r_d}_i > 0$: a weight parameter to avoid failure of numerical computation, ${r_q}_i \geq 0$: a weight parameter to increase mergin of the saturation  

`index`: $i$  
`u_min`: $u_{i, {\rm min}}$  
`u_max`: $u_{i, {\rm max}}$   
`dummy_weight`: ${r_d}_i > 0$  
`quadratic_weight` :  ${r_q}_i \geq 0$  
`saturation_param` = [`index`, `u_min`, `u_max`, `dummy_weight`, `quadratic_weight`]

In [ ]:
saturation_list = []
# saturation_list = [[index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    .., 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight]

## Set parameters for the solver

`T_f`, `alpha`: parameters for the length of the horizon $T(t)$: $T(t) = T_f (1 - e^{-\alpha t})$  
`horizon_divs`: the division number of the horzion for the numerical computation  
`finite_diff_step`: a step length of a finite difference approximations of hessian-vector products in C/GMRES  
`zeta`: a stabilization parameter for the continuation transformation   
`kmax`: the maximam number of the iteration of the GMRES  

In [ ]:
T_f = 
alpha = 
horizon_divs = 
finite_diff_step = 
zeta =
kmax = 

solver_params = slvprm.SolverParams(T_f, alpha, horizon_divs, finite_diff_step, zeta, kmax)

## Set parameters for the initialization of the solution  
`initial_guess`: The initial guess of the solution of the optimal control problem for initialization of the solution of NMPC.   
`tol_res`: The torelance residual of the solution of the optimal control problem for the initialization of the solution of NMPC. The Newton iteration terminates when the optimality error is less than `tol_res`.  
`max_itr`: The maxmum number of Newton iteration for the initialization of the solution of NMPC.  
`Lag_multiplier`: A optional parameter for `MultipleShootingCGMRESWithSaturation`. This is a part of the initial guess of the solution, the initial guess of the Lagrange multiplier with respect the constraints on the saturation function of the control input.

In [ ]:
initial_guess = []
tol_res = 
max_itr = 
Lag_multiplier = []

if solver_index == 1 or solver_index == 2:
    initialization_params = iniprm.InitializationParams(initial_guess, tol_res, max_itr)
else:
    initialization_params = iniprm.InitializationParams(initial_guess, tol_res, max_itr, Lag_multiplier)

## Set parameters for numerical simulation
`initial_time`: initial time of the numerical simulation  
`initial_state`: initial state vector of the system  
`simulation_time`: simulation time of the numerical simulation  
`sampling_time`: the sampling time of the numerical simulation

In [ ]:
initial_time = 
initial_state = []  
simulation_time = 
sampling_time = 

simulation_params = simprm.SimulationParams(initial_time, initial_state, simulation_time, sampling_time)

## Generate main.cpp and CMakeLists.txt

In [ ]:
if solver_index == 1 or solver_index == 2:
    gencpp.generate_main(solver_index, solver_params, initialization_params, simulation_params, model_name)
else:
    gencpp.generate_main(solver_index, solver_params, initialization_params, simulation_params, model_name, saturation_list)    

gencpp.generate_cmake(solver_index, model_name)
gencpp.generate_cmake_for_model(model_name)

## Build and run simulation
NOTE: if you use Windows OS and an error occurs in `cppexec.setCMake(model_name)`, you may solve that error by running codes which are commented out instead of the original codes

In [ ]:
cppexe.set_cmake(model_name)
cppexe.make_and_run(model_name)
# cppexe.remove_build_dir(model_name)
# cppexe.set_cmake(model_name, MSYS=True)
# cppexe.make_and_run(model_name)

## Plot the simulation results

In [ ]:
plot = simplt.SimulationPlottor(model_name)
plot.set_scales(2,5,2)
# plot.show_plots()
plot.save_plots()